# Useful Features

---

## Toolbox Functions [Code](https://github.com/FlorianShepherd/pandapower-youtube/blob/master/scripts/toolbox.py)

> Toolbox is like swiss knife of pandapower and can be used for different thingg. e.g. merge two grids, results are equal etc.

<a href="http://www.youtube.com/watch?feature=player_embedded&v=O74yw1FmmsM" target="_blank">
    <img 
        src="http://img.youtube.com/vi/O74yw1FmmsM/0.jpg" 
        alt="Easy plotting Youtube Video" width="240" height="180" border="10" 
    />
</a>

In [7]:
# imports
import pandapower.toolbox as tb
import pandapower.networks as nw
import pandapower as pp

In [11]:
net = nw.create_cigre_network_mv()
pp.runpp(net)
# print some power flow information
tb.lf_info(net)

In [16]:
net2 = nw.create_cigre_network_mv()
net2.load.drop(index=0, inplace=True)
# check if two nets are identical. You can also only check the results
tb.nets_equal(net, net2)

False

In [17]:
# Check if only results are equal
net = nw.create_cigre_network_mv()
net2 = nw.create_cigre_network_mv()
net2.load.drop(index=0, inplace=True)
pp.runpp(net)
pp.runpp(net2)

tb.nets_equal(net, net2, check_only_results=True)

Networks do not match in DataFrame(s): res_trafo, res_bus, res_load, res_ext_grid, res_line


False

In [13]:
# merge two power systems
net3 = tb.merge_nets(net, net2)
# drop some buses, drops also elements of those buses e.g drops also lines and loades connected to that bus
tb.drop_buses(net, buses=[5])
# creates a continuous index in net.bus starting at 10
tb.create_continuous_bus_index(net, start=10)

{0: 10,
 1: 11,
 2: 12,
 3: 13,
 4: 14,
 6: 15,
 7: 16,
 8: 17,
 9: 18,
 10: 19,
 11: 20,
 12: 21,
 13: 22,
 14: 23}

In [14]:
# get all line elements connected to buses
els = tb.get_connected_elements(net, "line", buses=[11])
print(els)
print(net.line.loc[els])

{0}
       name        std_type  from_bus  to_bus  ...   df  parallel  type  in_service
0  Line 1-2  CABLE_CIGRE_MV        11      12  ...  1.0         1    cs        True

[1 rows x 14 columns]


---

## Diagnostic Tool [Code](https://github.com/FlorianShepherd/pandapower-youtube/blob/master/scripts/pp_diagnostic.py) | [Documentation](https://github.com/e2nIEE/pandapower/blob/master/tutorials/diagnostic.ipynb)

> A power flow calculation on a pandapower network can fail to converge (or fail to run at all) for a vast variety of reasons, which often makes debugging difficult, annoying and time consuming. To help with that, the diagnostic function automatically checks pandapower networks for the most common issues leading to errors. It provides logging output and diagnoses with a controllable level of detail.

<a href="http://www.youtube.com/watch?feature=player_embedded&v=EA2pytl3MKg" target="_blank">
    <img 
        src="http://img.youtube.com/vi/EA2pytl3MKg/0.jpg" 
        alt="Easy plotting Youtube Video" width="240" height="180" border="10" 
    />
</a>

In [18]:
import pandapower as pp
import pandapower.plotting as plt

In [23]:
# defines a function that creates an example network, which will be used a lot in this tutorial
# run this code first in order for the examples to work

def faulty_example_network():

    net = pp.create_empty_network()

    pp.create_bus(net, name = "110 kV bar", vn_kv = 110, type = 'b', in_service = 'True')
    pp.create_bus(net, name = "20 kV bar", vn_kv = 20, type = 'b')
    pp.create_bus(net, name = "bus 2", vn_kv = 30, type = 'b')
    pp.create_bus(net, name = "bus 3", vn_kv = 20, type = 'b')
    pp.create_bus(net, name = "bus 4", vn_kv = 20, type = 'b')
    pp.create_bus(net, name = "bus 5", vn_kv = -20, type = 'b')
    pp.create_bus(net, name = "bus 6", vn_kv = 20, type = 'b')
    
    pp.create_ext_grid(net, 0, vm_pu = 1)

    pp.create_line(net, name = "line 0", from_bus = 1, to_bus = 2, length_km = 0, std_type = "NAYY 4x150 SE")
    pp.create_line(net, name = "line 1", from_bus = 2, to_bus = 3, length_km = 1, std_type = "NAYY 4x150 SE")
    pp.create_line(net, name = "line 2", from_bus = 3, to_bus = 4, length_km = 1, std_type = "NAYY 4x150 SE")
    pp.create_line(net, name = "line 3", from_bus = 4, to_bus = 5, length_km = 1, std_type = "NAYY 4x150 SE")
    pp.create_line(net, name = "line 4", from_bus = 5, to_bus = 6, length_km = 1, std_type = "NAYY 4x150 SE")
    pp.create_line(net, name = "line 5", from_bus = 6, to_bus = 1, length_km = 1, std_type = "NAYY 4x150 SE")

    pp.create_transformer_from_parameters(net, hv_bus = 1, lv_bus = 0, i0_percent= 0.038, pfe_kw = 11.6,
                                          vkr_percent = 0.322, sn_mva = 40.0, vn_lv_kv = 22.0,
                                          vn_hv_kv = 110.0, vk_percent = 17.8)

    pp.create_load(net, 2, p_mw = -1, q_mvar = 0.200, name = "load 0")
    pp.create_load(net, 3, p_mw = 1, q_mvar = 0.200, name = "load 1")
    pp.create_load(net, 4, p_mw = 1, q_mvar = 0.200, name = "load 2")
    pp.create_load(net, 5, p_mw = 1, q_mvar = 0.200, name = "load 3")
    pp.create_load(net, 6, p_mw = 1, q_mvar = 0.200, name = "load 4")

    pp.create_switch(net, bus = 1, element = 0, et = 'l')
    pp.create_switch(net, bus = 2, element = 0, et = 'l')
    pp.create_switch(net, bus = 2, element = 1, et = 'l')
    pp.create_switch(net, bus = 3, element = 1, et = 'l')
    pp.create_switch(net, bus = 3, element = 2, et = 'l')
    pp.create_switch(net, bus = 4, element = 2, et = 'l')
    pp.create_switch(net, bus = 4, element = 3, et = 'l', closed = False)
    pp.create_switch(net, bus = 5, element = 3, et = 'l')
    pp.create_switch(net, bus = 5, element = 4, et = 'l', closed = False)
    pp.create_switch(net, bus = 6, element = 4, et = 'l', closed = False)
    pp.create_switch(net, bus = 6, element = 5, et = 'l')
    pp.create_switch(net, bus = 1, element = 5, et = 'l')
    
    return net

In [26]:
def diag():
    net = faulty_example_network()
    # pp.runpp(net)
    # plt.simple_plot(net, plot_line_switches=True)
    pp.diagnostic(net)

diag()



_____________ PANDAPOWER DIAGNOSTIC TOOL _____________ 


 --------

Checking for elements without a connection to an external grid...

Disconnected section found, consisting of the following elements:
buses: [5]
switches: [7, 8]
lines: [3]
loads: [3]

SUMMARY: 5 disconnected element(s) found.

 --------

Checking for connections of different voltage levels...

line 0 connects bus 1: 20 kV bar (vn_kv = 20.0) and bus 2: bus 2 (vn_kv = 30.0)
line 1 connects bus 2: bus 2 (vn_kv = 30.0) and bus 3: bus 3 (vn_kv = 20.0)
line 3 connects bus 4: bus 4 (vn_kv = 20.0) and bus 5: bus 5 (vn_kv = -20.0)
line 4 connects bus 5: bus 5 (vn_kv = -20.0) and bus 6: bus 6 (vn_kv = 20.0)

SUMMARY: 4 element(s) that connect different voltage levels found.

 --------

Checking for impedance values close to zero...

line 0: r_ohm <= 0.001 or x_ohm <= 0.001
Power flow still does not converge after replacing implausible elements with switches.

SUMMARY: 1 element(s) with impedance values close to zero found.

 

---

## Read Your Data from Excel [Code](https://github.com/FlorianShepherd/pandapower-youtube/blob/master/scripts/read_data_excel.py) | [Documentation]()

<a href="http://www.youtube.com/watch?feature=player_embedded&v=QeOrLzb9T8o" target="_blank">
    <img 
        src="http://img.youtube.com/vi/QeOrLzb9T8o/0.jpg" 
        alt="Easy plotting Youtube Video" width="240" height="180" border="10" 
    />
</a>

In [1]:
import pandas as pd
import pandapower as pp

net = pp.create_empty_network()

# For loading buses from excel
df = pd.read_excel("./data_files/read_data_excel.xlsx", sheet_name="bus", index_col=0)
for idx in df.index:
    pp.create_bus(net, vn_kv=df.at[idx, "vn_kv"], )
# print(net.bus)

# for loading loads from excel
df = pd.read_excel("./data_files/read_data_excel.xlsx", sheet_name="load", index_col=0)
for idx in df.index:
    pp.create_load(net, bus=df.at[idx, "bus"], p_mw=df.at[idx, "p"])
# print(net.load)

# for slack buses from excel
df = pd.read_excel("./data_files/read_data_excel.xlsx", sheet_name="slack", index_col=0)
for idx in df.index:
    pp.create_ext_grid(net, bus=df.at[idx, "bus"], vm_pu=df.at[idx, "vm_pu"], 
                       va_degree=df.at[idx, "va_degree"])
# print(net.ext_grid)

# for line data from excel
df = pd.read_excel("./data_files/read_data_excel.xlsx", sheet_name="line", index_col=0)
for idx in df.index:
    pp.create_line_from_parameters(net, *df.loc[idx, :])
# print(net.line)

# for transformer from excel
df = pd.read_excel("./data_files/read_data_excel.xlsx", sheet_name="trafo", index_col=0)
for idx in df.index:
    pp.create_transformer(net, hv_bus=df.at[idx, "hv_bus"], lv_bus=df.at[idx, "lv_bus"], 
                          std_type=df.at[idx, "std_type"])

pp.runpp(net)
print(net.res_bus)

      vm_pu  va_degree     p_mw    q_mvar
0  1.000000   0.000000 -0.07507 -0.002683
1  0.996066  -0.649068  0.00000  0.000000
2  0.846274  -1.707654  0.03000  0.000000
3  0.767827  -2.394062  0.03000  0.000000


c:\Users\junai\Documents\Workspaces\Jupyter_Notebook\pandapower-youtube\venv\lib\site-packages\pandapower\results_branch.py:141: RuntimeWarning: divide by zero encountered in true_divide
  res_line_df["loading_percent"].values[:] = i_ka / i_max * 100


---